In [1]:
import numpy as np
import pandas as pd

In [2]:
DIR = './data'
MASKING_VER = 'masked_02_drop'

train = pd.read_csv(f'{DIR}/train_{MASKING_VER}.csv', index_col=0)

print(train.shape)
train.head()

(472970, 2)


,level,full_log
id,,
0,0,"<DATE> <TIME> localhost kibana: {""type"":""error..."
1,0,"<DATE> <TIME> localhost logstash: [<TS>,<NUM>]..."
2,0,"<DATE> <TIME> localhost kibana: {""type"":""error..."
3,0,"<DATE> <TIME> localhost kibana: {""type"":""error..."
4,1,type=SYSCALL msg=audit(<NUM>.<NUM>:<NUM>): arc...


In [3]:
train['level'].value_counts()

0    334065
1    132517
3      4140
5      2219
2        11
4        10
6         8
Name: level, dtype: int64

# 특수문자

In [4]:
train['first_word'] = train['full_log'].str.extract('^([^ ]+)(?: |$)')

In [5]:
train['first_word'].value_counts()

<DATE>           349575
type=SYSCALL     116496
level              2331
ossec:             1318
File                822
System              820
error:              459
type=AVC            369
oscap:              366
E:                  297
<YEAR>               31
NTFS                 25
Trojaned             15
Windows              13
--MARK--:            12
The                  11
juniper               4
type=USER_AVC         3
OpenSCAP              3
Name: first_word, dtype: int64

In [6]:
def show_contains(target, *, col='full_log', first_words=None, cnt=None):
    condition = train[col].str.contains(target)
    if first_words is not None:
        condition &= train['first_word'].isin(first_words)
    
    logs = train[condition][col][:cnt] if cnt is not None else train[condition][col]
    for log in logs:
        print(log, '\n')

## 공백

In [7]:
train['masked'] = train['full_log'].str.replace('\s+', ' ', regex=True)

## `][`

In [8]:
show_contains('\]\[', col='masked', cnt=5)

<DATE> <TIME> localhost logstash: [<TS>,<NUM>][INFO ][logstash.outputs.elasticsearch] retrying failed action with response code: <NUM> ({"type"=>"unavailable_shards_exception", "reason"=>"[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>][<NUM>] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>][<NUM>]] containing [<NUM>] requests]"}) 

<DATE> <TIME> localhost logstash: [<TS>,<NUM>][WARN ][logstash.outputs.elasticsearch] Attempted to resurrect connection to dead ES instance, but got an error. {:url=>"http://localhost:<NUM>/", :error_type=>LogStash::Outputs::ElasticSearch::HttpClient::Pool::HostUnreachableError, :error=>"Elasticsearch Unreachable: [http://localhost:<NUM>/][Manticore::SocketException] 연결이 거부됨 (Connection refused)"} 

<DATE> <TIME> localhost logstash: [<TS>,<NUM>][WARN ][logstash.outputs.elasticsearch] Attempted to resurrect connection to dead ES instance, but got an error. {:url=>"http://localhost:<NUM>/", :e

In [9]:
train['masked'] = train['masked'].str.replace('\]\[', '] [', regex=True)

In [10]:
for idx in train[train['full_log'].str.contains('\]\[')].index[:3]:
    print(train['full_log'][idx], '\n')
    print(train['masked'][idx], '\n')

<DATE> <TIME> localhost logstash: [<TS>,<NUM>][INFO ][logstash.outputs.elasticsearch] retrying failed action with response code: <NUM> ({"type"=>"unavailable_shards_exception", "reason"=>"[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>][<NUM>] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>][<NUM>]] containing [<NUM>] requests]"}) 

<DATE> <TIME> localhost logstash: [<TS>,<NUM>] [INFO ] [logstash.outputs.elasticsearch] retrying failed action with response code: <NUM> ({"type"=>"unavailable_shards_exception", "reason"=>"[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>] [<NUM>] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>] [<NUM>]] containing [<NUM>] requests]"}) 

<DATE> <TIME> localhost logstash: [<TS>,<NUM>][WARN ][logstash.outputs.elasticsearch] Attempted to resurrect connection to dead ES instance, but got an error. {:url=>"http://localhost:<NUM>/",

`,` 

In [11]:
show_contains('\S,\S', col='masked', cnt=5)

<DATE> <TIME> localhost kibana: {"type":"error","@timestamp":"<TS>","tags":["warning","stats-collection"],"pid":<NUM>,"level":"error","error":{"message":"No Living connections","name":"Error","stack":"Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:<NUM>:<NUM>)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:<NUM>:<NUM>)\n at process._tickCallback (internal/process/next_tick.js:<NUM>:<NUM>)"},"message":"No Living connections"} 

<DATE> <TIME> localhost logstash: [<TS>,<NUM>] [INFO ] [logstash.outputs.elasticsearch] retrying failed action with response code: <NUM> ({"type"=>"unavailable_shards_exception", "reason"=>"[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>] [<NUM>] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>] [<NUM>]] containing [<NUM>] requests]"}) 

<DATE> <TIME> localhost kibana: {"type":"error","@t

In [12]:
train['masked'] = train['masked'].str.replace(',', ' , ', regex=True)

In [13]:
for idx in train[train['full_log'].str.contains(',')].index[:3]:
    print(train['full_log'][idx], '\n')
    print(train['masked'][idx], '\n')

<DATE> <TIME> localhost kibana: {"type":"error","@timestamp":"<TS>","tags":["warning","stats-collection"],"pid":<NUM>,"level":"error","error":{"message":"No Living connections","name":"Error","stack":"Error: No Living connections\n    at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:<NUM>:<NUM>)\n    at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:<NUM>:<NUM>)\n    at process._tickCallback (internal/process/next_tick.js:<NUM>:<NUM>)"},"message":"No Living connections"} 

<DATE> <TIME> localhost kibana: {"type":"error" , "@timestamp":"<TS>" , "tags":["warning" , "stats-collection"] , "pid":<NUM> , "level":"error" , "error":{"message":"No Living connections" , "name":"Error" , "stack":"Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:<NUM>:<NUM>)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:<NUM>:<NUM>)

## 기타

`:`

In [14]:
show_contains('":"', col='masked', cnt=3)

<DATE> <TIME> localhost kibana: {"type":"error" , "@timestamp":"<TS>" , "tags":["warning" , "stats-collection"] , "pid":<NUM> , "level":"error" , "error":{"message":"No Living connections" , "name":"Error" , "stack":"Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:<NUM>:<NUM>)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:<NUM>:<NUM>)\n at process._tickCallback (internal/process/next_tick.js:<NUM>:<NUM>)"} , "message":"No Living connections"} 

<DATE> <TIME> localhost kibana: {"type":"error" , "@timestamp":"<TS>" , "tags":["warning" , "stats-collection"] , "pid":<NUM> , "level":"error" , "error":{"message":"No Living connections" , "name":"Error" , "stack":"Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:<NUM>:<NUM>)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:<NU

### =>

In [15]:
show_contains('=>', col='masked', cnt=3)

<DATE> <TIME> localhost logstash: [<TS> , <NUM>] [INFO ] [logstash.outputs.elasticsearch] retrying failed action with response code: <NUM> ({"type"=>"unavailable_shards_exception" ,  "reason"=>"[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>] [<NUM>] primary shard is not active Timeout: [1m] ,  request: [BulkShardRequest [[wazuh-alerts-audit-<NUM>.x-<NUM>.<NUM>.<NUM>] [<NUM>]] containing [<NUM>] requests]"}) 

<DATE> <TIME> localhost logstash: [<TS> , <NUM>] [WARN ] [logstash.outputs.elasticsearch] Attempted to resurrect connection to dead ES instance ,  but got an error. {:url=>"http://localhost:<NUM>/" ,  :error_type=>LogStash::Outputs::ElasticSearch::HttpClient::Pool::HostUnreachableError ,  :error=>"Elasticsearch Unreachable: [http://localhost:<NUM>/] [Manticore::SocketException] 연결이 거부됨 (Connection refused)"} 

<DATE> <TIME> localhost logstash: [<TS> , <NUM>] [WARN ] [logstash.outputs.elasticsearch] Attempted to resurrect connection to dead ES instance ,  but got an error. {:url=>"h

`=`

In [16]:
show_contains('\S=\S', first_words=set(train['first_word'].unique()).difference({'<DATE>', 'type=SYSCALL'}), cnt=3)

type=AVC msg=audit(<NUM>.<NUM>:<NUM>): avc:  denied  { execute } for  pid=<NUM> comm="sudo" name="agent_control" dev="dm-<NUM>" ino=<NUM> scontext=system_u:system_r:httpd_t:s0 tcontext=system_u:object_r:var_t:s0 tclass=file permissive=<NUM> type=AVC msg=audit(<NUM>.<NUM>:<NUM>): avc:  denied  { read open } for  pid=<NUM> comm="sudo" path="/var/esild/bin/agent_control" dev="dm-<NUM>" ino=<NUM> scontext=system_u:system_r:httpd_t:s0 tcontext=system_u:object_r:var_t:s0 tclass=file permissive=<NUM> type=AVC msg=audit(<NUM>.<NUM>:<NUM>): avc:  denied  { execute_no_trans } for  pid=<NUM> comm="sudo" path="/var/esild/bin/agent_control" dev="dm-<NUM>" ino=<NUM> scontext=system_u:system_r:httpd_t:s0 tcontext=system_u:object_r:var_t:s0 tclass=file permissive=<NUM> type=SYSCALL msg=audit(<NUM>.<NUM>:<NUM>): arch=<NUM> syscall=<NUM> success=yes exit=<NUM> a0=<NUM> a1=<NUM> a2=<NUM> a3=<NUM> items=<NUM> ppid=<NUM> pid=<NUM> auid=<NUM> uid=<NUM> gid=<NUM> euid=<NUM> suid=<NUM> fsuid=<NUM> egid=<NUM> 

In [17]:
show_contains('\S=\S', first_words={'type=SYSCALL'}, cnt=3)

type=SYSCALL msg=audit(<NUM>.<NUM>:<NUM>): arch=<NUM> syscall=<NUM> success=yes exit=<NUM> a0=<NUM> a1=<NUM> a2=<NUM> a3=<NUM> items=<NUM> ppid=<NUM> pid=<NUM> auid=<NUM> uid=<NUM> gid=<NUM> euid=<NUM> suid=<NUM> fsuid=<NUM> egid=<NUM> sgid=<NUM> fsgid=<NUM> tty=(none) ses=<NUM> comm="date" exe="/usr/bin/date" subj=system_u:system_r:sysstat_t:s0-s0:c0.c1023 key="audit-wazuh-r" type=CWD msg=audit(<NUM>.<NUM>:<NUM>):  cwd="/root" type=PATH msg=audit(<NUM>.<NUM>:<NUM>): item=<NUM> name="/etc/ld.so.cache" inode=<NUM> dev=<NUM>:<NUM> mode=<NUM> ouid=<NUM> ogid=<NUM> rdev=<NUM>:<NUM> obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(<NUM>.<NUM>:<NUM>): proctitle=<NUM> 

type=SYSCALL msg=audit(<NUM>.<NUM>:<NUM>): arch=<NUM> syscall=<NUM> success=yes exit=<NUM> a0=<NUM> a1=<NUM> a2=<NUM> a3=<NUM> items=<NUM> ppid=<NUM> pid=<NUM> auid=<NUM> uid=<NUM> gid=<NUM> euid=<NUM> suid=<NUM> fsuid=<NUM> egid=<NUM> sgid=<NUM> fsgid=<NUM> tty=(none) ses=<NUM> comm="ps" exe=

In [18]:
show_contains('(?<=localhost sudo:).+?\S=\S', first_words={'<DATE>'}, cnt=3)

<DATE> <TIME> localhost sudo: pam_unix(sudo:session): session opened for user root by (uid=<NUM>) 

<DATE> <TIME> localhost sudo:    root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/bin/ping -c <NUM> <IP> 

<DATE> <TIME> localhost sudo: pam_unix(sudo:session): session opened for user root by (uid=<NUM>) 



# apply masking

In [19]:
TARGET_COLUMN = 'full_log'
PATTERNS = [('\]\[', '] ['), 
            (',', ' , '), 
            ('\s+', ' ')]

def apply_masking(df):
    for pat, repl in PATTERNS:
        df[TARGET_COLUMN] = df[TARGET_COLUMN].str.replace(pat, repl, regex=True)

In [20]:
MASKING_VER = 'masked_02'

train = pd.read_csv(f'{DIR}/train_{MASKING_VER}_drop.csv', index_col=0)
validation = pd.read_csv(f'{DIR}/validation_{MASKING_VER}.csv', index_col=0)
test = pd.read_csv(f'{DIR}/test_{MASKING_VER}.csv', index_col=0)

train.shape, validation.shape, test.shape

C:\Users\ASIA-11\anaconda3\envs\prj_final\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


((472970, 2), (3, 1), (1418916, 1))

In [21]:
FILES = {'train': train, 'validation': validation, 'test': test}

for file, df in FILES.items():
    apply_masking(df)
    df.to_csv(f'./data/{file}_masked_03.csv')